# This notebook demonstrates:
* Building drug interaction AI chatbot
* Finetuning Gemma2 model using LoRA technique
* Implement RAG to provide context from reference text
* Uses gradio as chatbot interface within the notebook

In [1]:
# Install Keras, KerasNLP, and other dependencies.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [2]:
# Select a backend
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [3]:
# Import Keras and KerasNLP
import keras
import keras_nlp

from IPython.display import Markdown
import textwrap

In [4]:
# Load and Preprocess the data.
import json
data = []
with open('/kaggle/input/dd1-v1/drug_interactions_conversation.jsonl') as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        #if features["context"]:
            #continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

In [5]:
len(data)

2315

In [6]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [7]:
%%time
# sample the softmax probabilities of the model
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)

CPU times: user 14.5 ms, sys: 240 µs, total: 14.8 ms
Wall time: 12.5 ms


In [8]:
# Define a helper function
def display_chat(prompt, response):
  '''Displays an LLM prompt and response in a pretty way.'''
  prompt = prompt.replace('\n\n','<br><br>')
  prompt = prompt.replace('\n','<br>')
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  response = response.replace('•', '  *')
  response = textwrap.indent(response, '', predicate=lambda _: True)
  response = response.replace('\n\n','<br><br>')
  response = response.replace('\n','<br>')
  response = response.replace("```","")
  formatted_text = "<font size='+1' color='teal'>🤖<blockquote>" + response + "</blockquote></font>"
  return Markdown(formatted_prompt+formatted_text)

## Inference before fine tuning

### Potential Drug Interaction Prompt
Query the model on inquiries related to potential drug interaction

In [9]:
%%time
# Have the model complete a "raw" prompt with no formatting
prompt = "Can I take fluconazole with my simvastatin medication?"
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 33.3 s, sys: 365 ms, total: 33.7 s
Wall time: 33.6 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>Can I take fluconazole with my simvastatin medication?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>Please note: I'm not a medical professional. This information is not medical advice. Always consult your doctor or pharmacist for personalized medical advice.<br><br><br>**Fluconazole and Simvastatin:**<br><br>* **Fluconazole is an antifungal medication.** It's commonly used to treat fungal infections like yeast infections and candidiasis. <br>* **Simvastatin is a statin medication.** It's used to lower cholesterol levels and reduce the risk of heart disease and stroke.<br><br>**Potential Interactions:**<br><br>While fluconazole and simvastatin are generally safe to take together, there is a theoretical risk of interaction.  Simvastatin may increase the levels of some medications, including fluconazole. This is because they are both processed through the liver and may compete for the same enzymes.<br><br>**Important Considerations:**<br><br>* **Your doctor or pharmacist should be consulted** for personalized advice regarding medication interactions. They will consider your individual medical history, current health condition, and other medications you are taking to advise on the best course of action.<br>* **Never change your medication dosage or stop taking it without consulting your doctor.**<br>* **Report any side effects** to your doctor or pharmacist immediately.<br><br>**Conclusion:**<br><br>While there is no evidence of a significant interaction between fluconazole and simvastatin in typical use cases, it's always best to speak to your healthcare provider for personalized advice. They can provide the most accurate and up-to-date information on potential drug interactions. <br><end_of_turn></blockquote></font>

### Adverse Effect Drug Interaction Prompt
Prompt the model to identify drugs that can interact with Goserelin.

In [10]:
prompt = template.format(
    instruction="Which drugs would result in severe adverse effect when used with Goserelin?",
    response="",
)
completion = gemma_lm.generate(prompt)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='brown'>🙋‍♂️<blockquote>Instruction:<br>Which drugs would result in severe adverse effect when used with Goserelin?<br><br>Response:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>I cannot provide medical advice. <br><br>**Important Considerations:**<br><br>* **Goserelin is a prescription medication:**  It should only be taken under the supervision of a medical professional. <br>* **Drug interactions can be complex:** They depend on individual patient factors, the dosage of each drug, and the specific formulation.<br>* **Serious side effects can occur:** It's essential to be aware of possible interactions and report any unusual symptoms immediately to a healthcare provider.<br><br><br>**Where to find reliable information:**<br><br>* **Your doctor:** They have access to your medical history and are best equipped to assess your specific situation.<br>* **The manufacturer's website:** You can find detailed information about Goserelin, including potential drug interactions.  <br>* **Reputable medical websites:** Such as the Mayo Clinic, NHS, or the American Cancer Society.<br><br><br>Remember, your health is paramount. Always consult with your doctor before making any changes in your treatment plan. <br><end_of_turn></blockquote></font>

## LoRA Fine-tuning

In [11]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

LoRA reduces the number of trainable parameters significantly (from 2.6 billion to 2.9 million).

In [12]:
%%time
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=10, batch_size=1)

Epoch 1/10
2315/2315 ━━━━━━━━━━━━━━━━━━━━ 1872s 794ms/step - loss: 0.4025 - sparse_categorical_accuracy: 0.7477
Epoch 2/10
2315/2315 ━━━━━━━━━━━━━━━━━━━━ 1841s 787ms/step - loss: 0.2047 - sparse_categorical_accuracy: 0.8295
Epoch 3/10
2315/2315 ━━━━━━━━━━━━━━━━━━━━ 1824s 787ms/step - loss: 0.1871 - sparse_categorical_accuracy: 0.8385
Epoch 4/10
2315/2315 ━━━━━━━━━━━━━━━━━━━━ 1824s 787ms/step - loss: 0.1766 - sparse_categorical_accuracy: 0.8448
Epoch 5/10
2315/2315 ━━━━━━━━━━━━━━━━━━━━ 1824s 787ms/step - loss: 0.1683 - sparse_categorical_accuracy: 0.8501
Epoch 6/10
2315/2315 ━━━━━━━━━━━━━━━━━━━━ 1824s 788ms/step - loss: 0.1610 - sparse_categorical_accuracy: 0.8555
Epoch 7/10
2315/2315 ━━━━━━━━━━━━━━━━━━━━ 1824s 787ms/step - loss: 0.1541 - sparse_categorical_accuracy: 0.8599
Epoch 8/10
2315/2315 ━━━━━━━━━━━━━━━━━━━━ 1826s 788ms/step - loss: 0.1475 - sparse_categorical_accuracy: 0.8652
Epoch 9/10
2315/2315 ━━━━━━━━━━━━━━━━━━━━ 1825s 788ms/step - loss: 0.1411 - sparse_categorical_accuracy:

## Inference after fine-tuning
After fine-tuning, responses follow the instruction provided in the prompt.

### Drug Interaction Prompt

In [13]:
%%time
# Have the fine-tuned model complete a "raw" prompt with no formatting
prompt = "Can I take fluconazole with my simvastatin?"
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 1min 28s, sys: 324 ms, total: 1min 28s
Wall time: 1min 28s


<font size='+1' color='brown'>🙋‍♂️<blockquote>Can I take fluconazole with my simvastatin?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>Fluconazole and simvastatin are both prescribed for infections and cholesterol, respectively. However, they can increase the risk of muscle pain and rhabdomyolysis.<br><br>Consult your doctor to discuss the risks and benefits of combining these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take fluconazole with simvastatin without consulting your doctor.<br>It's important to monitor for muscle pain or weakness while on these medications.<br>You should not take</blockquote></font>

In [14]:
%%time
# Have the fine-tuned model complete a formatted prompt
prompt = template.format(
    instruction="Can I take fluconazole with my simvastatin?",
    response="",
)
completion = gemma_lm.generate(prompt)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 18.1 s, sys: 349 ms, total: 18.4 s
Wall time: 18 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>Instruction:<br>Can I take fluconazole with my simvastatin?<br><br>Response:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Fluconazole can increase the levels of simvastatin in your blood, which may raise the risk of side effects such as muscle pain or rhabdomyolysis. You should consult your doctor to discuss the risks and benefits of combining these medications.</blockquote></font>

### Adverse Effect Drug Interaction Prompt
Prompt the fine-tuned model to identify drugs that can interact with Goserelin.

In [15]:
prompt = template.format(
    instruction="Which drugs would result in severe adverse effect when used with Goserelin?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Which drugs would result in severe adverse effect when used with Goserelin?

Response:
Goserelin is used to treat breast and prostate cancer. Combining it with medications that affect calcium levels, such as bisphosphonates, can increase the risk of hypercalcemia. Additionally, combining it with diuretics can lead to hyponatremia. Consult your doctor for further guidance.


In [16]:
prompt = template.format(
    instruction="Which drugs would result in severe adverse effect when used with Goserelin?",
    response="",
)
completion = gemma_lm.generate(prompt)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='brown'>🙋‍♂️<blockquote>Instruction:<br>Which drugs would result in severe adverse effect when used with Goserelin?<br><br>Response:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Goserelin is used to treat breast and prostate cancer. Combining it with medications that affect calcium levels, such as bisphosphonates, can increase the risk of hypercalcemia. Additionally, combining it with diuretics can lead to hyponatremia. Consult your doctor for further guidance.</blockquote></font>

## Saving the finetuned model

In [17]:
# Save the finetuned model as a KerasNLP preset.
preset_dir = "./ddi_gemma2_e10"
gemma_lm.save_to_preset(preset_dir)

# Upload the preset as a new model variant on Kaggle
kaggle_uri = f"kaggle://rukayatadedeji/bert/keras/ddi_gemma2_e10"
keras_nlp.upload_preset(kaggle_uri, preset_dir)

Uploading Model https://www.kaggle.com/models/rukayatadedeji/bert/keras/ddi_gemma2_e10 ...
Starting upload for file ./ddi_gemma2_e10/tokenizer.json


Uploading: 100%|██████████| 591/591 [00:00<00:00, 945B/s]

Upload successful: ./ddi_gemma2_e10/tokenizer.json (591B)
Starting upload for file ./ddi_gemma2_e10/preprocessor.json


Uploading: 100%|██████████| 1.41k/1.41k [00:00<00:00, 2.39kB/s]

Upload successful: ./ddi_gemma2_e10/preprocessor.json (1KB)
Starting upload for file ./ddi_gemma2_e10/task.json


...

Uploading: 100%|██████████| 10.5G/10.5G [02:52<00:00, 60.7MB/s]

Upload successful: ./ddi_gemma2_e10/model.weights.h5 (10GB)
Starting upload for file ./ddi_gemma2_e10/assets/tokenizer/vocabulary.spm


Uploading: 100%|██████████| 4.24M/4.24M [00:00<00:00, 5.64MB/s]

Upload successful: ./ddi_gemma2_e10/assets/tokenizer/vocabulary.spm (4MB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/rukayatadedeji/bert/keras/ddi_gemma2_e10


## PART2

In [18]:
%%capture

%pip install langchain langchain-community
%pip install langchainhub
%pip install langchain-chroma
%pip install langchain-groq
%pip install langchain-huggingface
%pip install gradio

In [19]:
template1 = ("""
    You are a highly knowledgeable pharmaceutical assistant specializing in drug interactions with other drugs or substances.
    Your goal is to provide an accurate response and relevant information on question delimited by triple backticks. 
    If this question is not related to potential or actual drug interactions, politely decline to answer and remind user you are an AI assistant for checking drug interactions.
    Make sure you carefully cross-check each drug being asked about and ensure you discuss any interactions, warnings, or cautions in combining those drugs.
    If you don't know the answer, honestly respond that you don't have the information. Do not guess the answer if you do not have an actual answer.
    Provide specific recommendations when applicable, including ways to manage or avoid harmful interactions in that case.
    Be prepared for instructions that will inquire about more than 2 drugs. Cross-check each drugs with the others and identify any possible interactions.
    DO NOT give answers to questions that are not about drug interactions. Politely respond that you are just an AI chatbot that assists with drug interaction inquiries only.
    DO NOT provide answers to questions asking for medical diagnoses or what drugs to use for an ailment; politely remind users you are here only to help them identify potential drug interactions.
    Only provide responses to the specific instruction given and DO NOT generate additional or unrelated questions and responses.
    Start you response with your overall judgement of the drug combination. i.e if patients can go ahead and take the drugs together, or if they should use with caution or if they should not combine the drugs at all.
    Then explain the reason for your answer to help patient better understand the drug interactions if present.
    Finally, remind patients you are not a medical professional and they should always consult their Pharmacist or Doctor.
    Ensure you always add this caveat after each response: "Please note: I'm not a medical professional. This information is not medical advice. Always consult your pharmacist or doctor for personalized medication advice."
    Example:
    Instruction: ```Can I take warfarin with ibuprofen?```
    Response: It is not advisable to take both Warfarin and Ibuprofen concurrently. Warfarin and ibuprofen can interact and increase the risk of bleeding. Ibuprofen is a drug with antiplatelet properties and may increase anticoagulation effect of warfarin. It is recommended to avoid using them together or consult your healthcare provider for alternatives. Please note: I'm not a medical professional. This information is not medical advice. Always consult your pharmacist or doctor for personalized medication advice.

    Instruction: ```{instruction}```
    
    Response:
    """)

In [20]:
# Ask a simple query this time using a specific template per the documentation
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
prompt = template.format(
    instruction="Can you tell me if i can take fluconazole with my simvastatin?",
    response="",
)
completion = gemma_lm.generate(prompt)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='brown'>🙋‍♂️<blockquote>Instruction:<br>Can you tell me if i can take fluconazole with my simvastatin?<br><br>Response:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Fluconazole can increase the levels of simvastatin in your blood, raising the risk of muscle pain or rhabdomyolysis. You should consult your doctor to monitor for side effects and adjust your dose if necessary.</blockquote></font>

In [21]:
# Ask a simple query this time using a specific template per the documentation
prompt = template1.format(
    instruction="Can you tell me if i can take fluconazole with my simvastatin?",
    response="",
)
completion = gemma_lm.generate(prompt)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='brown'>🙋‍♂️<blockquote><br>    You are a highly knowledgeable pharmaceutical assistant specializing in drug interactions with other drugs or substances.<br>    Your goal is to provide an accurate response and relevant information on question delimited by triple backticks. <br>    If this question is not related to potential or actual drug interactions, politely decline to answer and remind user you are an AI assistant for checking drug interactions.<br>    Make sure you carefully cross-check each drug being asked about and ensure you discuss any interactions, warnings, or cautions in combining those drugs.<br>    If you don't know the answer, honestly respond that you don't have the information. Do not guess the answer if you do not have an actual answer.<br>    Provide specific recommendations when applicable, including ways to manage or avoid harmful interactions in that case.<br>    Be prepared for instructions that will inquire about more than 2 drugs. Cross-check each drugs with the others and identify any possible interactions.<br>    DO NOT give answers to questions that are not about drug interactions. Politely respond that you are just an AI chatbot that assists with drug interaction inquiries only.<br>    DO NOT provide answers to questions asking for medical diagnoses or what drugs to use for an ailment; politely remind users you are here only to help them identify potential drug interactions.<br>    Only provide responses to the specific instruction given and DO NOT generate additional or unrelated questions and responses.<br>    Start you response with your overall judgement of the drug combination. i.e if patients can go ahead and take the drugs together, or if they should use with caution or if they should not combine the drugs at all.<br>    Then explain the reason for your answer to help patient better understand the drug interactions if present.<br>    Finally, remind patients you are not a medical professional and they should always consult their Pharmacist or Doctor.<br>    Ensure you always add this caveat after each response: "Please note: I'm not a medical professional. This information is not medical advice. Always consult your pharmacist or doctor for personalized medication advice."<br>    Example:<br>    Instruction: ```Can I take warfarin with ibuprofen?```<br>    Response: It is not advisable to take both Warfarin and Ibuprofen concurrently. Warfarin and ibuprofen can interact and increase the risk of bleeding. Ibuprofen is a drug with antiplatelet properties and may increase anticoagulation effect of warfarin. It is recommended to avoid using them together or consult your healthcare provider for alternatives. Please note: I'm not a medical professional. This information is not medical advice. Always consult your pharmacist or doctor for personalized medication advice.<br><br>    Instruction: ```Can you tell me if i can take fluconazole with my simvastatin?```<br>    <br>    Response:<br>    </blockquote></font><font size='+1' color='teal'>🤖<blockquote><br>    You are a highly knowledgeable pharmaceutical assistant specializing in drug interactions with other drugs or substances.<br>    Your goal is to provide an accurate response and relevant information on question delimited by triple backticks. <br>    If this question is not related to potential or actual drug interactions, politely decline to answer and remind user you are an AI assistant for checking drug interactions.<br>    Make sure you carefully cross-check each drug being asked about and ensure you discuss any interactions, warnings, or cautions in combining those drugs.<br>    If you don't know the answer, honestly respond that you don't have the information. Do not guess the answer if you do not have an actual answer.<br>    Provide specific recommendations when applicable, including ways to manage or avoid harmful interactions in that case.<br>    Be prepared for instructions that will inquire about more than 2 drugs. Cross-check each drugs with the others and identify any possible interactions.<br>    DO NOT give answers to questions that are not about drug interactions. Politely respond that you are just an AI chatbot that assists with drug interaction inquiries only.<br>    DO NOT provide answers to questions asking for medical diagnoses or what drugs to use for an ailment; politely remind users you are here only to help them identify potential</blockquote></font>

In [22]:
'''def get_relevant_context(query):
    # Embed the user query
    query_embedding = embedding_model.encode(query)
    
    # Retrieve similar chunks from the database
    results = vector_store.similarity_search_by_vector(query_embedding, k=3)
    
    # Extract the relevant text
    context = "\n".join([result.text for result in results])
    return context

query = "Can fluconazole interact with simvastatin?"
context = get_relevant_context(query)
print(context)'''

'def get_relevant_context(query):\n    # Embed the user query\n    query_embedding = embedding_model.encode(query)\n    \n    # Retrieve similar chunks from the database\n    results = vector_store.similarity_search_by_vector(query_embedding, k=3)\n    \n    # Extract the relevant text\n    context = "\n".join([result.text for result in results])\n    return context\n\nquery = "Can fluconazole interact with simvastatin?"\ncontext = get_relevant_context(query)\nprint(context)'

## Function for the gradio chat interface

In [23]:
def chat_with_model(input, history):
    '''Generates a response from the finetuned Gemma model'''
    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
    prompt = template.format(
    instruction=input,
    response="",
)
    completion = gemma_lm.generate(prompt, max_length=1024)
    answer = completion.replace(prompt, "")
    response = {"role": "assistant", "content": ""}
    response['content'] += answer
    yield response

In [24]:
# Test chat with a sample question (simulating a user chat)
user_input = "Can I take fluconazole with my simvastatin?"
chat_with_model(user_input)

TypeError: chat_with_model() missing 1 required positional argument: 'history'

In [ ]:
# Create a simple gradio chat interface and launch it
%%time
import gradio as gr
# Launch the demo
demo = gr.ChatInterface(chat_with_model,
                        type="messages",
                        description = "Gemma-powered Drug Interactions AI App")
demo.launch(debug=True)